In [1]:
# --- Install dependencies ---
!pip install gradio opencv-python-headless matplotlib tqdm pytransform3d


In [8]:
# Camera_Calibration_UI.ipynb

import gradio as gr
import os
import json
import subprocess
import shlex
import time
import numpy as np
import cv2
from calibration.chessboard import find_corners_in_folder
from calibration.core import calibrate, undistort_image
from calibration.calib_io import save_calibration_json, load_calibration_json
from calibration.overlay import overlay_axes_on_calibration_images
from calibration.simple_pose_viz import simple_pose_visualization

# -------------------------------------------------------------------
# Helper functions for Gradio
# -------------------------------------------------------------------

def save_uploaded_images(files, save_dir="data/images"):
    """Save uploaded .jpeg/.jpg files to the project images folder."""
    os.makedirs(save_dir, exist_ok=True)
    saved = []
    for f in files:
        fname = os.path.basename(f)
        path = os.path.join(save_dir, fname)
        os.replace(f, path)  # move temp file into project folder
        saved.append(path)
    return f"✅ Saved {len(saved)} images to {save_dir}", save_dir



def run_calibration_via_script(images_dir, cols, rows, square_size, auto_rational=True, min_improve_pct=3.0, min_improve_px=0.05):
    """
    Run the repository's calibration script as a subprocess and return summary + model selection.
    Returns: (summary_text, out_json_path, sample_image_path_or_none, model_selection_json_str)
    """
    images_dir = str(images_dir) if images_dir else "data/images"
    # match the script default output location unless user-configurable later
    out_json = os.path.join("data", "results", "calibration.json")

    cmd = (
        f'python scripts/run_calibration.py '
        f'--images_dir "{images_dir}" '
        f'--cols {int(cols)} --rows {int(rows)} --square_size {float(square_size)} '
        f'--out_json "{out_json}" '
        f'--min_improve_pct {float(min_improve_pct)} --min_improve_px {float(min_improve_px)}'
    )
    # Ensure undistort preview uses a wide, natural alpha and centers principal point
    cmd += ' --undistort_alpha 0.85 --undistort_center_pp'

    if not bool(auto_rational):
        cmd += ' --no_auto_rational'

    # Run subprocess
    try:
        proc = subprocess.run(shlex.split(cmd), capture_output=True, text=True)
    except Exception as e:
        return f"Failed to run calibration script: {e}", None, None, ""

    if proc.returncode != 0:
        # include stdout/stderr for debugging
        return f"Calibration script failed (rc={proc.returncode})\nSTDOUT:\n{proc.stdout}\nSTDERR:\n{proc.stderr}", None, None, ""

    # Wait briefly for the JSON file to appear
    for _ in range(50):
        if os.path.exists(out_json):
            break
        time.sleep(0.1)

    if not os.path.exists(out_json):
        return "Calibration completed but output JSON not found.", None, None, ""

    try:
        calib = load_calibration_json(out_json)
    except Exception as e:
        return f"Calibration JSON found but failed to load: {e}", out_json, None, ""

    # Build a full human summary similar to original UI
    summary = (
        f"RMS reprojection error: {calib['rms']:.4f}\n"
        f"Image size: {calib['image_size']}\n"
        f"Intrinsic matrix K:\n{np.array(calib['K'])}\n\n"
        f"Distortion coefficients:\n{np.array(calib['dist'])}\n"
    )

    model_info = calib.get('model_selection') or {}
    model_selection_str = json.dumps(model_info, indent=2) if model_info else "{}"

    # sample image: try to return first image from images_dir as a convenience
    sample_img_path = None
    try:
        files = [os.path.join(images_dir, f) for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        sample_img_path = files[0] if files else None
    except Exception:
        sample_img_path = None

    return summary, out_json, sample_img_path, model_selection_str



def preview_undistortion(calibration_json, image_path):
    """Create side-by-side undistortion preview (before/after)."""
    if calibration_json is None or not os.path.exists(calibration_json):
        return "❌ Calibration JSON not found. Run calibration first.", None
    
    if image_path is None or not os.path.exists(image_path):
        return "❌ Please select an image file.", None
    
    try:
        # Load calibration data
        calib = load_calibration_json(calibration_json)
        K, dist = calib["_K_np"], calib["_dist_np"]
        
        # Load and undistort image
        img = cv2.imread(image_path)
        if img is None:
            return "❌ Could not load image file.", None
            
        # Use the chosen preview undistortion settings (alpha=0.85, center_pp=True)
        und, _ = undistort_image(img, K, dist, balance=0.85, center_pp=True)
        
        # Create side-by-side comparison
        h, w = img.shape[:2]
        
        # Add text labels
        img_labeled = img.copy()
        und_labeled = und.copy()
        
        # Add "Original" and "Undistorted" labels
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img_labeled, "Original", (10, 30), font, 1, (0, 255, 0), 2)
        cv2.putText(und_labeled, "Undistorted", (10, 30), font, 1, (0, 255, 0), 2)
        
        # Combine side by side
        side_by_side = np.hstack([img_labeled, und_labeled])
        
        # Save preview
        os.makedirs("data/results", exist_ok=True)
        preview_path = "data/results/undistort_preview.jpg"
        cv2.imwrite(preview_path, side_by_side)
        
        return f"✅ Undistortion preview created! Original vs Undistorted comparison.", preview_path
        
    except Exception as e:
        return f"❌ Error creating preview: {str(e)}", None



def generate_axes_overlays(calibration_json, images_dir, output_dir="axes_results", max_images=8):
    try:
        paths = overlay_axes_on_calibration_images(calibration_json, images_dir, output_dir, max_images=max_images)
        return f"Created {len(paths)} overlay images", paths  # return all, not just 3
    except Exception as e:
        return f"Error generating overlays: {e}", None



def generate_pose_visualization(calibration_json, output_dir="pose_results"):
    try:
        # Use the simple pose visualization
        figs = simple_pose_visualization(calibration_json, output_dir, show_plot=False)
        out_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith(".png")]
        return f"Generated {len(out_files)} visualization plots", out_files
    except Exception as e:
        return f"Error generating poses: {e}", None

# -------------------------------------------------------------------
# Build Gradio Interface
# -------------------------------------------------------------------

with gr.Blocks() as demo:
    gr.Markdown("# 📷 Camera Calibration with Gradio UI")
    
    with gr.Tab("1. Upload Images"):
        uploads = gr.File(
            label="Upload Chessboard Images (.jpeg or .jpg)",
            file_types=[".jpeg", ".jpg"],  # accept both
            file_count="multiple",         # allow multiple uploads
            type="filepath"                # return file paths
        )
        upload_btn = gr.Button("Save Uploaded Images")
        upload_status = gr.Textbox(label="Upload Status")
        images_dir = gr.Textbox(label="Images Folder", value="data/images", interactive=False)
        
        upload_btn.click(save_uploaded_images, inputs=[uploads], outputs=[upload_status, images_dir])
    
    with gr.Tab("2. Calibration"):
        cols = gr.Number(label="Columns (inner corners)", value=9)
        rows = gr.Number(label="Rows (inner corners)", value=6)
        square_size = gr.Number(label="Square size (mm)", value=22.0)

        auto_rational = gr.Checkbox(label="Auto model selection (enabled by default)", value=True)
        min_improve_pct = gr.Number(label="Min improve % (rational vs baseline)", value=3.0)
        min_improve_px = gr.Number(label="Min improve px (absolute)", value=0.05)

        calib_btn = gr.Button("Run Calibration")
        calib_output = gr.Textbox(label="Calibration Results")
        calib_json = gr.File(label="Calibration JSON", interactive=False)
        sample_img = gr.Image(label="Sample Image Used", type="filepath")
        model_info = gr.Textbox(label="Model selection (JSON)", lines=10)
        
        calib_btn.click(
            run_calibration_via_script, 
            inputs=[images_dir, cols, rows, square_size, auto_rational, min_improve_pct, min_improve_px],
            outputs=[calib_output, calib_json, sample_img, model_info]
        )
    
    with gr.Tab("3. Undistortion Preview"):
        img_in = gr.Image(label="Choose Image", type="filepath")
        undist_btn = gr.Button("Preview Undistortion")
        undist_status = gr.Textbox(label="Status")
        undist_img = gr.Image(label="Preview")
        
        undist_btn.click(
            preview_undistortion,
            inputs=[calib_json, img_in],
            outputs=[undist_status, undist_img]
        )
    
    with gr.Tab("4. Axes Overlays"): 
        overlay_btn = gr.Button("Generate Axes Overlays (8 images)")
        overlay_status = gr.Textbox(label="Status")
        overlay_imgs = gr.Gallery(label="Overlay Samples", columns=4, height="auto")

        overlay_btn.click(
            generate_axes_overlays,
            inputs=[calib_json, images_dir],
            outputs=[overlay_status, overlay_imgs]
        )

    with gr.Tab("5. Camera Pose Visualization"):
        pose_btn = gr.Button("Generate 3D/2D Pose Visualizations")
        pose_status = gr.Textbox(label="Status")
        pose_imgs = gr.Gallery(label="Pose Plots", columns=2, height="auto")

        pose_btn.click(
            generate_pose_visualization,
            inputs=[calib_json],
            outputs=[pose_status, pose_imgs]
        )

    demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7867
* Running on public URL: https://96d488c4922662ec33.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
* Running on public URL: https://96d488c4922662ec33.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
